In [ ]:
import numpy as np
import imageio as imio
import os
import torch
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt
import random
import statistics
import copy
import math
import heapq
import time
from google.colab.patches import cv2_imshow
from PIL import Image, ImageDraw
from scipy.spatial import KDTree
from scipy.optimize import minimize
from sklearn.cluster import DBSCAN
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

xfeat = torch.hub.load('verlab/accelerated_features', 'XFeat', pretrained = True, top_k = 4096)
#Load some example images


/usr/local/lib/python3.11/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/verlab/accelerated_features/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://github.com/verlab/accelerated_features/raw/main/weights/xfeat.pt" to /root/.cache/torch/hub/checkpoints/xfeat.pt
100%|██████████| 5.96M/5.96M [00:00<00:00, 30.9MB/s]


In [ ]:
class CurveConcatenationLine:
    def __init__(self):
        """Initialization"""
        #self.img1_path = img1_path
        #self.img2_path = img2_path

    def distance(self,p1, p2):

        return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

    def find_nearest_below(self,current, points):

        _, (x1, y1) = current
        candidates = [p for p in points if p[1][1] > y1]

        if not candidates:
            return None
        return min(candidates, key=lambda p: new_curveconcatenationline.distance((x1, y1), p[1]))

    def find_path(self, points):

      path_ids = []
      path_points = []
      points = sorted(points, key=lambda x: x[1])
      current = points[0]
      path_ids.append(current[0])
      path_points.append(current[1])
      points.remove(current)

      while points:

          next_point = new_curveconcatenationline.find_nearest_below(current, points)
          if next_point is None:
              break

          path_ids.append(next_point[0])
          path_points.append(next_point[1])
          points.remove(next_point)
          current = next_point

      return path_ids, path_points

    def draw_points_on_image(self,points, image_path1, image_path):

        image = Image.open(image_path).convert("RGB")
        draw = ImageDraw.Draw(image)

        for x, y in points:

            if 0 <= x < image.width and 0 <= y < image.height:
                draw.ellipse((x - 3, y - 3, x + 3, y + 3), fill="red")
            else:
                print(f"Point ({x}, {y}) is out of bounds.")


        image.show()


        image.save(image_path1)

    def labelling_points(self,points):

        return [[i + 1, point] for i, point in enumerate(points)]
    def tracer_lignes_et_points_cv(self, image, points, nom_fichier='/content/sample_data/test.jpg'):

        image = image.copy()
        if image is None:
            print("Error: Unable to load image.")
            return

        hauteur, largeur, _ = image.shape

        centre_x = largeur // 2
        centre_y = hauteur // 2

        for point in points:
            cv2.circle(image, (int(point[0]), int(point[1])), 5, (0, 0, 255), -1)

        cv2.line(image, (centre_x, 0), (centre_x, hauteur), (0, 255, 0), 2)
        cv2.line(image, (0, centre_y), (largeur, centre_y), (0, 255, 0), 2)



    def getting_points_img2_corresponding(self,label_matched_kp2,path_ids):
        tuples_recuperes = []

        for pos in path_ids:

            if 1 <= pos <= len(label_matched_kp2):
                tuples_recuperes.append(label_matched_kp2[pos - 1][1])

        return tuples_recuperes


    def find_closest_points(self, points1, points2):

        closest_top_left = min(points1, key=lambda p: (p[0], p[1]))
        corresponding_top_left = points2[points1.index(closest_top_left)]


        closest_bottom_right = max(points1, key=lambda p: (p[0], p[1]))
        corresponding_bottom_right = points2[points1.index(closest_bottom_right)]

        closest_left = sorted(points1, key=lambda p: p[0])
        corresponding_left = [points2[points1.index(p)] for p in closest_left]

        closest_right = sorted(points1, key=lambda p: p[0], reverse=True)
        corresponding_right = [points2[points1.index(p)] for p in closest_right]

        closest_bottom = sorted(points1, key=lambda p: p[1], reverse=True)
        corresponding_bottom = [points2[points1.index(p)] for p in closest_bottom]

        latest_point = points1[-1]
        latest_point_corresponding = points2[-1]
        return latest_point, latest_point_corresponding,closest_left, corresponding_left, closest_top_left, corresponding_top_left, closest_right[-1], corresponding_right,closest_bottom[-1], corresponding_bottom,closest_bottom_right, corresponding_bottom_right

    def resize_mask_gen_2(self,mask1,mask2,image1_path, image2_path, closest_left, corresponding_left, latest_point,latest_point_corresponding,rx):

        image1 = cv2.imread(image1_path)
        image2 = cv2.imread(image2_path)
        print("latest_point:",latest_point)
        mask2 = cv2.imread(mask2, cv2.IMREAD_GRAYSCALE)
        mask1 = cv2.imread(mask1, cv2.IMREAD_GRAYSCALE)
        ry = int(round(latest_point)) - int(round(latest_point_corresponding))
        rycp=ry

        if ry > 0:
            image2 = np.pad(image2, ((ry, 0), (0, 0), (0, 0)), mode='constant', constant_values=0)
            mask2 = np.pad(mask2, ((ry,0), (0, 0)), mode='constant', constant_values=0)

            image1 = np.pad(image1, ((0, ry), (0, 0), (0, 0)), mode='constant', constant_values=0)
            mask1 = np.pad(mask1, ((0, ry), (0, 0)), mode='constant', constant_values=0)
        elif ry < 0:

            ry = abs(ry)

            image1 = np.pad(image1, ((ry, 0), (0, 0), (0, 0)), mode='constant', constant_values=0)
            mask1 = np.pad(mask1, ((ry,0), (0, 0)), mode='constant', constant_values=0)

            image2 = np.pad(image2, ((0, ry), (0, 0), (0, 0)), mode='constant', constant_values=0)
            mask2 = np.pad(mask2, ((0, ry), (0, 0)), mode='constant', constant_values=0)


        if mask1 is None or mask1.size == 0:
            print("mask1 is empty or None")
        if mask2 is None or mask2.size == 0:
            print("mask2 is empty or None")

        return image2,mask1,mask2, rycp


    def New_translation(self,image1_path, image2_path,closest_left, corresponding_left, mask_path,rx):

            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            image1 = cv2.imread(image1_path, cv2.IMREAD_UNCHANGED)
            image2 = cv2.imread(image2_path, cv2.IMREAD_UNCHANGED)

            if image1.shape[0] != image2.shape[0]:
                raise ValueError("The two images must be the same height.")

            if image1.shape[2] == 3:
                image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2BGRA)
            if image2.shape[2] == 3:
                image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2BGRA)

            height, width1, _ = image1.shape
            _, width2, _ = image2.shape

            if rx > 0:

                black_band = np.zeros((height, rx, 4), dtype=np.uint8)
                adjusted_image1 = np.hstack((image1, black_band))
                adjusted_image2 = np.hstack((black_band, image2))

                padding_right = np.zeros((height, rx), dtype=mask.dtype)
                mask = np.hstack((mask, padding_right))

            elif rx == 0:
                adjusted_image1 = image1
                adjusted_image2 = image2

            mask_normalized = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)[1]

            if adjusted_image1.shape[:2] != adjusted_image2.shape[:2] or adjusted_image1.shape[:2] != mask_normalized.shape:
                raise ValueError("The dimensions of the images and the mask do not match after adjustment.")


            result = adjusted_image1.copy()
            result[mask_normalized == 0] = adjusted_image2[mask_normalized == 0]
            widthF, heightF=result.shape[:2]

            return result


    def apply_mask(self, output_path,image_path, mask_path):

          image = cv2.imread(image_path)
          mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

          if image is None or mask is None:

              return

          _, binary_mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

          inverted_mask = cv2.bitwise_not(binary_mask)

          masked_image = cv2.bitwise_and(image, image, mask=inverted_mask)

          cv2.imwrite(output_path, masked_image)

          return masked_image

    def apply_mask_without_inversion(self,image, mask_path, output_path):

          mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

          if image is None or mask is None:
              print("Error loading image or mask.")
              return

          _, binary_mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

          inverted_mask = cv2.bitwise_not(binary_mask)


          masked_image = cv2.bitwise_and(image, image, mask=mask)

          # Save the result
          cv2.imwrite(output_path, masked_image)
          return masked_image


    def generate_triangles(self, points):
        triangles = []
        connections = []

        for i in range(0, len(points) - 1, 2):
            a = points[i]
            b = points[i + 1]
            distance_ab = new_curveconcatenationline.distance(a, b)

            y_difference = abs(a[1] - b[1])

            if distance_ab >= 4 and y_difference >= 4:

                c = (a[0], b[1])
                d = new_curveconcatenationline.distance(a, c)
                e = new_curveconcatenationline.distance(c, b)

                triangles.append({'a': a, 'b': b, 'c': c, 'd': d, 'e': e})

                if i + 2 < len(points):
                    a_prime = points[i + 2]
                    t = new_curveconcatenationline.distance(b, a_prime)
                    connections.append(t)


        return triangles, connections


    def draw_triangles(self,image_path, triangles, connections):

        img = Image.open(image_path)
        draw = ImageDraw.Draw(img)

        for triangle in triangles:
            a = triangle['a']
            b = triangle['b']
            c = triangle['c']
            draw.polygon([a, b, c], outline='black', fill=None)

        for i in range(len(connections)):
            if i + 1 < len(triangles):
                a = triangles[i]['b']
                b = triangles[i + 1]['a']
                draw.line([a, b], fill='blue', width=2)


    def plot_lines_on_image(self,img,points,case):
        if img is None:
            raise ValueError("The image could not be loaded. Check the image path.")

        height, width, _ = img.shape
        all_points = []

        mask = np.ones((height, width), dtype=np.uint8) * 255

        first_point = min(points, key=lambda p: p[1])

        last_point = max(points, key=lambda p: p[1])
        start_point = (int(round(first_point[0])), 0)
        end_point = (int(round(first_point[0])), int(round(first_point[1])))
        all_points.extend(new_curveconcatenationline.get_vertical_trajectory_points(start_point, int(round(first_point[1]))))
        check =new_curveconcatenationline.get_vertical_trajectory_points(start_point,int(round(first_point[1])))

        for i in range(0, len(points) - 1, 2):
            a = (int(round(points[i][0])), int(round(points[i][1])))
            b = (int(round(points[i + 1][0])), int(round(points[i + 1][1])))
            c = (int((a[0] + b[0]) // 2), int(min(a[1], b[1])))

            all_points.extend(new_curveconcatenationline.get_hypotenuse_points(a, c))
            all_points.extend(new_curveconcatenationline.get_hypotenuse_points(b, c))

            cv2.line(img, a, c, (0, 255, 255), 2)
            cv2.line(img, b, c, (0, 255, 255), 2)
            if i + 2 < len(points):
                a_prime = (int(round(points[i + 2][0])), int(round(points[i + 2][1])))
                cv2.line(img, b, a_prime, (0, 255, 255))
        vertical_end_1080 = (int(round(last_point[0])), height)
        all_points.extend(new_curveconcatenationline.get_vertical_trajectory_points((int(round(last_point[0])), int(round(last_point[1]))), height))

        for j in range(len(all_points) - 1):
            cv2.line(img, all_points[j], all_points[j + 1], (0, 255, 255), 2)

        if case=="im1":

          for y in range(height):
              row = img[y, :, :]
              yellow_pixel_index = np.where((row[:, 0] == 0) & (row[:, 1] == 255) & (row[:, 2] == 255))[0]

              if yellow_pixel_index.size > 0:

                  first_yellow_index = yellow_pixel_index[0]

                  if first_yellow_index < width:
                      row[first_yellow_index:] = [0, 0, 0]
                      mask[y, first_yellow_index:] = 0
        if case =="im2":

          for y in range(height):
              row = img[y, :, :]
              yellow_pixel_index = np.where((row[:, 0] == 0) & (row[:, 1] == 255) & (row[:, 2] == 255))[0]

              if yellow_pixel_index.size > 0:

                  first_yellow_index = yellow_pixel_index[0]

                  if first_yellow_index > 0:
                      row[:first_yellow_index] = [0, 0, 0]
                      mask[y, :first_yellow_index] = 0

        mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

        return all_points, img, mask,points,check

    def get_vertical_trajectory_points(self,start_point, end_y):

        x = start_point[0]
        trajectory_points = []

        for y in range(start_point[1], end_y + 1):
            trajectory_points.append((x, y))

        return trajectory_points

    def get_hypotenuse_points(self,point1, point2):
        points = []
        x1, y1 = point1
        x2, y2 = point2

        dist = int(np.linalg.norm(np.array(point2) - np.array(point1)))

        if dist == 0:
            return [point1]
        for i in range(dist + 1):
            x = int(round(x1 + (x2 - x1) * i / dist))
            y = int(round(y1 + (y2 - y1) * i / dist))
            points.append((x, y))

        return points

    def draw_dotted_line(self,img, pt1, pt2, color, thickness=1, gap=5):

        dist = int(round(np.linalg.norm(np.array(pt2) - np.array(pt1))))
        for i in range(0, dist, gap * 2):
            start = (
                int(round(pt1[0] + (pt2[0] - pt1[0]) * i / dist)),
                int(round(pt1[1] + (pt2[1] - pt1[1]) * i / dist))
            )
            end = (
                int(round(pt1[0] + (pt2[0] - pt1[0]) * (i + gap) / dist)),
                int(round(pt1[1] + (pt2[1] - pt1[1]) * (i + gap) / dist))
            )
            cv2.line(img, start, end, color, thickness)

    def translate_points(self,points, ry):

        return [(x, y + ry) for (x, y) in points]

    def calculate_new_image_size(self,points1, points2, image1_size, image2_size):

        width1, height1 = image1_size
        width2, height2 = image2_size

        max_width = 0

        for (x1, y1), (x2, y2) in zip(points1, points2):
            left_width = x1
            right_width = width2 - x2
            total_width = left_width + right_width
            max_width = max(max_width, total_width)

        new_height = max(y1 for _, y1 in points1) + 1

        return max_width, new_height


    def horizontal_cut(self,img, liste1):
        segments = []
        img_height, img_width = img.shape[:2]
        for i in range(len(liste1)):

            upper = 0 if i == 0 else int(liste1[i - 1][1])
            lower = int(liste1[i][1])
            if upper < lower <= img_height:
                segment = img[upper:lower, 0:img_width]
                segments.append(segment)
            else:
                print("invalid data for slicing")
        if liste1 and liste1[-1][1] < img_height:
            lower = int(liste1[-1][1])
            segment = img[lower:img_height, 0:img_width]
            segments.append(segment)

        return segments

    def list_dimensions_list_points_adapt(self,segments1, liste1):
        list_dimensions = []
        list_points_adapt = []

        for i, segment in enumerate(segments1):
            if isinstance(segment, np.ndarray):
                height, width = segment.shape[:2]
                list_dimensions.append((width, height))
            else:
                raise ValueError(f"The segment at index {i} is not a valid image object.")

        if liste1:
            first_width, first_height = liste1[0]
            list_points_adapt.append((first_width, 0))
            list_points_adapt.append((first_width, first_height))

            for i in range(len(liste1) - 1):
                current_width, current_height = liste1[i]
                next_width, next_height = liste1[i + 1]
                list_points_adapt.append((current_width, 0))
                list_points_adapt.append((next_width, abs(next_height - current_height)))
            last_height = list_dimensions[-1][1]
            last_width = liste1[-1][0]
            list_points_adapt.append((last_width, 0))
            list_points_adapt.append((last_width, last_height))
        return list_dimensions, list_points_adapt

    def add_black_band(self,image, height_diff, position='top'):
        if height_diff <= 0:
            return image
        band = np.zeros((int(height_diff), image.shape[1], 3), dtype=np.uint8)  # Créer une bande noire
        if position == 'top':
            new_image = np.vstack([band, image])
        elif position == 'bottom':
            new_image = np.vstack([image, band])
        return new_image

    def adjust_intermediate_height(self,image,original_width,original_height,target_height):

        if image.shape[0] == target_height:
            return image

        aspect_ratio = original_width / original_height
        new_width = int(target_height * aspect_ratio)

        resized_image = cv2.resize(image, (original_width, target_height), interpolation=cv2.INTER_LINEAR)

        return resized_image

    def update_coordinates(self,original_coords, height_diff):
        """Updates the coordinates with the new height."""
        x, y = original_coords
        new_coords = (x, y + abs(height_diff))

        return new_coords

    def create_mask(self,image_shape, pt1, pt2, position):
        mask = np.zeros(image_shape[:2], dtype=np.uint8)
        height, width = image_shape[:2]

        if position == 'left':
            points = np.array([[0, 0], [int(round(pt1[0])), int(round(pt1[1]))], [int(round(pt2[0])), int(round(pt2[1]))], [0, height]])
        elif position == 'right':
            points = np.array([[width, 0], [int(round(pt1[0])), int(round(pt1[1]))], [int(round(pt2[0])), int(round(pt2[1]))], [width, height]])
        elif position == 'middle':
            points = np.array([[int(round(pt1[1])), int(round(pt1[0]))], [int(round(pt2[1])), int(round(pt2[0]))], [int(round(pt2[1])), height], [int(round(pt1[0])), height]])
        else:
            raise ValueError("Invalid position specified. Choose from 'left', 'right', or 'middle'.")

        cv2.fillPoly(mask, [points], 255)
        return mask

    def blend_middle_regions(self,cp1_middle, cp2_middle):
        min_height = min(cp1_middle.shape[0], cp2_middle.shape[0])
        min_width = min(cp1_middle.shape[1], cp2_middle.shape[1])

        cp1_middle_resized = cv2.resize(cp1_middle, (min_width, min_height))
        cp2_middle_resized = cv2.resize(cp2_middle, (min_width, min_height))

        cp1_middle_float = cp1_middle_resized.astype(np.float32)
        cp2_middle_float = cp2_middle_resized.astype(np.float32)

        height, width, _ = cp1_middle_resized.shape
        alpha_cp1 = np.linspace(1, 0, width).reshape(1, width, 1).repeat(height, axis=0)
        alpha_cp2 = 1 - alpha_cp1

        blended_middle = (cp1_middle_float * alpha_cp1 + cp2_middle_float * alpha_cp2).astype(np.uint8)
        cv2.imwrite("blended_middle.jpg",blended_middle)
        return blended_middle


    def sub_concatenation_middle(self,img1, img2, points1, points2):

        if img1 is None or img2 is None:
            raise ValueError("One or both input images are None")
        mask_cp1 = new_curveconcatenationline.create_mask(img1.shape, points1[0], points1[1], 'left')
        mask_cp2 = new_curveconcatenationline.create_mask(img2.shape, points2[0], points2[1], 'middle')
        mask_cp3 = new_curveconcatenationline.create_mask(img2.shape, points2[0], points2[1], 'right')
        cp1 = cv2.bitwise_and(img1, img1, mask=mask_cp1)
        cp2 = cv2.bitwise_and(img2, img2, mask=mask_cp2)
        cp3 = cv2.bitwise_and(img2, img2, mask=mask_cp3)

        if points1[0][0]<points1[1][0]:
            cp1 = cp1[:, :int(round(points1[0][0]))]
            cp1_middle = img1[:, int(round(points1[0][0])):int(round(points1[1][0]))]
            cp2_middle = img2[:, int(round(points2[0][0])):int(round(points2[1][0]))]
            cp3 = cp3[:, int(round(points2[1][0])):]

        if points1[0][0]>=points1[1][0]:
            cp1 = cv2.bitwise_and(img1, img1, mask=mask_cp1)
            cp1 = img1[:, :int(round(points1[1][0]))]

            cp1_middle = img1[:, int(round(points1[1][0])):int(round(points1[0][0]))]
            cp2_middle = img2[:, int(round(points2[1][0])):int(round(points2[0][0]))]
            cp3 = img2[:, int(round(points2[0][0])):]

        blended_middle = new_curveconcatenationline.blend_middle_regions(cp1_middle, cp2_middle)
        result = np.hstack((cp1, blended_middle, cp3))

        return result

    def is_near_white(self,pixel, threshold=220):
        return all(c >= threshold for c in pixel)

    def blend_middle_regions_first_last(self, cp1_middle, cp2_middle, tolerance=70):
        min_height = min(cp1_middle.shape[0], cp2_middle.shape[0])
        min_width = min(cp1_middle.shape[1], cp2_middle.shape[1])

        cp1_resized = cv2.resize(cp1_middle, (min_width, min_height))
        cp2_resized = cv2.resize(cp2_middle, (min_width, min_height))

        cp1_middle_float = cp1_resized.astype(np.float32)
        cp2_middle_float = cp2_resized.astype(np.float32)
        intensity_cp1 = np.mean(cp1_middle_float)
        intensity_cp2 = np.mean(cp2_middle_float)
        alpha_cp1 = np.linspace(1, 0, min_width).reshape(1, min_width, 1).repeat(min_height, axis=0)
        alpha_cp2 = 1 - alpha_cp1

        blended_middle = np.where(
            abs(intensity_cp1 - intensity_cp2) <= tolerance,
            (cp1_middle_float * alpha_cp1 + cp2_middle_float * alpha_cp2),
            np.minimum(cp1_middle_float, cp2_middle_float)
        )

        return blended_middle.astype(np.uint8)

    def sub_concatenation_first_last(self,img1, img2, points1, points2):

        if img1 is None or img2 is None:
            raise ValueError("Could not read one or both images")

        height1, width1 = img1.shape[:2]
        height2, width2 = img2.shape[:2]

        overlap_width = 30
        x1_start = int(round(points1[0][0] - overlap_width))
        x1_end = int(round(points1[0][0] + overlap_width))
        x2_start = int(round(points2[0][0] - overlap_width))
        x2_end = int(round(points2[0][0] + overlap_width))
        x1_start = max(0, x1_start)
        x1_end = min(width1, x1_end)
        x2_start = max(0, x2_start)
        x2_end = min(width2, x2_end)
        cp1 = img1[:, :x1_start]
        cp1_middle = img1[:, x1_start:x1_end]
        cp2_middle = img2[:, x2_start:x2_end]
        cp3 = img2[:, x2_end:]
        min_height = min(cp1_middle.shape[0], cp2_middle.shape[0])
        cp1_middle_resized = cv2.resize(cp1_middle, (cp1_middle.shape[1], min_height))
        cp2_middle_resized = cv2.resize(cp2_middle, (cp2_middle.shape[1], min_height))
        blended_middle = new_curveconcatenationline.blend_middle_regions_first_last(cp1_middle_resized, cp2_middle_resized)
        cp1 = cv2.resize(cp1, (cp1.shape[1], min_height))
        cp3 = cv2.resize(cp3, (cp3.shape[1], min_height))
        result = np.hstack((cp1, blended_middle, cp3))
        return result

    def concatenate_images(self,segment1, segment2, points1, points2, is_first=False, is_last=False):
        n=0
        try:
            shift_first=0
            if is_first:
                height_diff = points1[1][1] - points2[1][1]
                shift_first= height_diff
                if height_diff < 0:
                    segment1 = new_curveconcatenationline.add_black_band(segment1, abs(height_diff), position='top')
                else:
                    segment2 = new_curveconcatenationline.add_black_band(segment2, abs(height_diff), position='top')
                concatenated = new_curveconcatenationline.sub_concatenation_first_last(segment1, segment2, points1, points2)
                n=n+1
            elif is_last:
                height_diff = points2[1][1] - points1[1][1]
                shift_first= height_diff
                if height_diff < 0:
                    segment2 = new_curveconcatenationline.add_black_band(segment2, abs(height_diff), position='bottom')
                else:
                    segment1 = new_curveconcatenationline.add_black_band(segment1, abs(height_diff), position='bottom')
                concatenated = new_curveconcatenationline.sub_concatenation_first_last(segment1, segment2, points1, points2)

            else:
                height1, height2 = segment1.shape[0], segment2.shape[0]
                width1, width2 = segment1.shape[1], segment2.shape[1]
                target_height = max(height1, height2)
                height_diff1 = points1[1][1] - points2[1][1]
                if height1 < target_height:
                    segment1 = new_curveconcatenationline.adjust_intermediate_height(segment1, width1,height1,target_height)
                    points1[1] = new_curveconcatenationline.update_coordinates(points1[1], height_diff1)
                if height2 < target_height:
                    segment2 = new_curveconcatenationline.adjust_intermediate_height(segment2,width2,height2,target_height)
                    points2[1] = new_curveconcatenationline.update_coordinates(points2[1], height_diff1)
                if height_diff1==0:
                    segment1=segment1
                    segment2=segment2

                concatenated = new_curveconcatenationline.sub_concatenation_middle(segment1, segment2, points1, points2)
                n=n+1
            return concatenated,shift_first

        except Exception as e:
            print(f"Error in concatenating images: {e}")
            return None

    def process_segments(self,segments1, segments2, points_list1, points_list2):
          results_list = []

          for i, (segment1, segment2) in enumerate(zip(segments1, segments2)):

              print(f"Segments processing{i}.................................................")
              print("segment1.shape:",segment1.shape)
              print("segment2.shape:",segment2.shape)
              if i * 2 + 1 >= len(points_list1) or i * 2 + 1 >= len(points_list2):
                  results_list.append(None)
                  continue

              points1 = points_list1[i * 2: (i + 1) * 2]
              points2 = points_list2[i * 2: (i + 1) * 2]

              is_first = (i == 0)
              is_last = (i == len(segments1) - 1)
              shift_first_l=[]
              try:
                  result,shift_first =new_curveconcatenationline.concatenate_images(segment1, segment2, points1, points2, is_first, is_last)
                  if result is None:
                      print("result.shape:",result.shape)
                  results_list.append(result)
                  shift_first_l.append(shift_first)
              except Exception as e:
                  results_list.append(None)
                  shift_first_l.append(None)
          return results_list,shift_first_l[0]


    def resize_images_to_average_width(self,images):

        average_width = int(sum(img.shape[1] for img in images) / len(images))
        resized_images = [
            cv2.resize(img, (average_width, int(round(img.shape[0] * (average_width / img.shape[1])))))
            for img in images
        ]
        return resized_images

    def concatenate_images_vertically(self,images):

        max_width = max(img.shape[1] for img in images)
        resized_images = [cv2.copyMakeBorder(img, 0, 0, 0, max_width - img.shape[1], cv2.BORDER_CONSTANT, value=(0, 0, 0)) for img in images]

        return np.vstack(resized_images)

    def process_images_vertically(self,images, output_dir):

        if not isinstance(images, list) or not all(isinstance(img, np.ndarray) for img in images):
            raise ValueError("The “images” parameter must be a list of image matrices (OpenCV).")

        resized_images = new_curveconcatenationline.resize_images_to_average_width(images)

        result = new_curveconcatenationline.concatenate_images_vertically(resized_images)
        output_path = os.path.join(output_dir, f'{len(images) + 3}.png')
        cv2.imwrite(output_path, result)
        return result

    def adjust_image_portion(image, x, target_width, original_width):

        height, width = image.shape[:2]
        x = int(round(x))
        adjustment_width = int(round(target_width - (original_width - x)))

        if adjustment_width > 0:

            extended_part = np.zeros((height, adjustment_width, 3), dtype=image.dtype)
            adjusted_image = np.hstack((image[:, :x], extended_part, image[:, x:]))
        elif adjustment_width < 0:
            new_width = width + adjustment_width
            adjusted_image = np.hstack((image[:, :x], image[:, x:new_width]))
        else:
            adjusted_image = image

        return adjusted_image

    def process_segmentsV(self, segments, list1):
        print("Number of segments:", len(segments))

        if not segments:
            print("Error: the segments list is empty.")
            return None, None
        valid_segments = []
        for i, img in enumerate(segments):
            if self.is_null_image(img):
                print(f"The segments[{i}] image is null or invalid. It will be ignored.")
            else:
                valid_segments.append(img)

        if not valid_segments:
            print("Error: no valid image found.")
            return None, None
        expected_length = 2 * (len(valid_segments) - 1) + 1
        widths = [img.shape[1] for img in valid_segments]
        width_difference = max(widths) - min(widths)
        resized_images = new_curveconcatenationline.resize_images_to_average_width(valid_segments)
        result_vertical = np.vstack(resized_images)
        return valid_segments, result_vertical

    def is_null_image(self,img):
        if img is None:
            return True
        if not isinstance(img, np.ndarray):
            return True
        if img.size == 0:
            return True
        if img.max() == img.min():
            return True
        return False


    def process_and_concatenate_images_vertically(self,images, output_dir):
        for i, img in enumerate(images):
            print(f"Image {i}: Type={type(img)}, Shape={getattr(img, 'shape', None)}")
        if not isinstance(images, list) or not all(isinstance(img, np.ndarray) for img in images):
            raise ValueError("The “images” parameter must be a list of image matrices (OpenCV).")

        average_width = int(sum(img.shape[1] for img in images) / len(images))

        resized_images = [
            cv2.resize(img, (average_width, int(round(img.shape[0] * (average_width / img.shape[1])))))
            for img in images
        ]

        max_width = max(img.shape[1] for img in resized_images)

        padded_images = [
            cv2.copyMakeBorder(img, 0, 0, 0, max_width - img.shape[1], cv2.BORDER_CONSTANT, value=(0, 0, 0))
            for img in resized_images
        ]
        result = np.vstack(padded_images)
        output_path = os.path.join(output_dir, f'{len(images) + 3}.png')
        cv2.imwrite(output_path, result)

        return result

    def resize_image_portion(self,image, x, target_portion_width):

        height, width = image.shape[:2]
        x = int(round(x))
        target_portion_width = int(round(target_portion_width))
        if x >= width:
            return image
        left_part = image[:, :x]
        right_part = image[:, x:]
        resized_right_part = cv2.resize(right_part, (target_portion_width, height))
        adjusted_image = np.hstack((left_part, resized_right_part))
        return adjusted_image

    def filter_lists(self,L1, L2):
        filtered_L1 = [L1[0]]
        filtered_L2 = [L2[0]]
        for i in range(len(L1) - 1):
            x1_current, y1_current = filtered_L1[-1]
            x1_next, y1_next = L1[i + 1]
            x2_current, y2_current = filtered_L2[-1]
            x2_next, y2_next = L2[i + 1]

            if (x1_current > x1_next and x2_current > x2_next) or (x1_current < x1_next and x2_current < x2_next):
                filtered_L1.append((x1_next, y1_next))
                filtered_L2.append((x2_next, y2_next))

        return filtered_L1, filtered_L2

    def remove_duplicates(self,L1, L2):
        i = 0
        while i < len(L1) - 1:
            if L1[i][0] == L1[i + 1][0]:
                del L1[i + 1]
                del L2[i + 1]
            else:
                i += 1
        return L1, L2

    def sort_and_filter(self,L1, L2):

        combined = list(zip(L1, L2))
        combined.sort(key=lambda x: x[1][1])
        filtered = []
        seen = set()
        for l1, l2 in combined:
            if l2[1] not in seen:
                filtered.append((l1, l2))
                seen.add(l2[1])
        L1_filtered, L2_filtered = zip(*filtered)
        return L1_filtered, L2_filtered

    def get_scharr_intensities(self, image, points):
        if len(image.shape) == 3 and image.shape[2] == 3:
            image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            image_gray = image.copy()
        grad_x = cv2.Scharr(image_gray, cv2.CV_64F, 1, 0)
        grad_y = cv2.Scharr(image_gray, cv2.CV_64F, 0, 1)
        magnitude = np.sqrt(grad_x**2 + grad_y**2)
        angle = np.arctan2(grad_y, grad_x) * 180 / np.pi
        intensities = {}
        directions = {}
        list_features_we_gets_intensity = []
        max_top_intensity = -1.0
        max_bottom_intensity = -1.0
        top_point = None
        bottom_point = None

        def bilinear_interpolation(x, y, img):

            x0, y0 = int(x), int(y)
            x1, y1 = min(x0 + 1, img.shape[1] - 1), min(y0 + 1, img.shape[0] - 1)
            dx, dy = x - x0, y - y0
            I00, I10 = img[y0, x0], img[y0, x1]
            I01, I11 = img[y1, x0], img[y1, x1]
            I_top = (1 - dx) * I00 + dx * I10
            I_bottom = (1 - dx) * I01 + dx * I11
            I_final = (1 - dy) * I_top + dy * I_bottom

            return float(I_final)

        for (x, y) in points:
            if 0 <= x < image_gray.shape[1] and 0 <= y < image_gray.shape[0]:
                intensity = bilinear_interpolation(x, y, magnitude)
                direction = bilinear_interpolation(x, y, angle)

                intensities[(x, y)] = intensity
                directions[(x, y)] = direction
                list_features_we_gets_intensity.append((x, y))
                if top_point is None or (y < top_point[1]) or (y == top_point[1] and intensity > max_top_intensity):
                    max_top_intensity = intensity
                    top_point = (x, y)
                if bottom_point is None or (y > bottom_point[1]) or (y == bottom_point[1] and intensity > max_bottom_intensity):
                    max_bottom_intensity = intensity
                    bottom_point = (x, y)
            else:
                intensities[(x, y)] = None
                directions[(x, y)] = None

        return intensities, list_features_we_gets_intensity, directions, top_point, bottom_point


    def compute_shift(self,point1, point2):
        K=np.array([[1446.4595928607794, 0.0, 1112.4044030939442], [0.0, 1454.7765408094683, 589.8928895307112], [0.0,
        0.0, 1.0]])
        proj1 = new_curveconcatenationline.transform_cylindrical_projection(point1, K)  # Project point1
        proj2 = new_curveconcatenationline.transform_cylindrical_projection(point2, K)  # Project point2
        shift_x = proj2[0] - proj1[0]
        shift_y = proj2[1] - proj1[1]

        return (shift_x, shift_y)



    def compute_intensity_differences(self,intensities1, intensities2):

        differences = []

        if len(intensities1) != len(intensities2):
            raise ValueError("Both intensity dictionaries must have the same number of points.")

        for ((x1, y1), intensity1), ((x2, y2), intensity2) in zip(intensities1.items(), intensities2.items()):
            intensity_diff = intensity1 - intensity2
            differences.append([(x1, y1), (x2, y2), intensity_diff])

        return differences

    def filter_and_match(self,differences):

        if not differences:
          return [], [], None
        intensity_diffs = [diff[2] for diff in differences]
        first_quartile = np.percentile(intensity_diffs, 50)

        filtered_diffs = [(point1, point2, intensity_diff) for point1, point2, intensity_diff in differences if intensity_diff <= first_quartile]
        if filtered_diffs:
            mean_of_quartile = np.mean([diff[2] for diff in filtered_diffs])
        else:
            mean_of_quartile = None

        match1 = [point1 for point1, _, _ in filtered_diffs]
        match2 = [point2 for _, point2, _ in filtered_diffs]

        return match1, match2

    def index_longest_list(self,lists):
        if not lists:
            return None

        longest_index = 0
        longest_length = len(lists[0])

        for index, sublist in enumerate(lists):
            if len(sublist) > longest_length:
                longest_length = len(sublist)
                longest_index = index

        return longest_index


    def affine_warping(self, image1, points1, points2):
        if len(points1) >= 3:

            points1 = np.array(points1, dtype=np.float32).reshape(-1, 2)
            points2 = np.array(points2, dtype=np.float32).reshape(-1, 2)

            matrice_affine, inliers = cv2.estimateAffine2D(points1, points2)

            if matrice_affine is not None:

                h, w = image1.shape[:2]
                points_corners = np.float32([[0, 0], [w, 0], [0, h], [w, h]]).reshape(-1, 1, 2)

                ones = np.ones((4, 1, 1), dtype=np.float32)
                points_corners_h = np.concatenate([points_corners, ones], axis=2)

                transformed_corners = cv2.transform(points_corners_h, matrice_affine)
                x_min, y_min = np.int32(transformed_corners.min(axis=0).flatten())
                x_max, y_max = np.int32(transformed_corners.max(axis=0).flatten())

                translation = np.array([[1, 0, -x_min], [0, 1, -y_min]], dtype=np.float32)
                matrice_affine = np.vstack([matrice_affine, [0, 0, 1]])
                matrice_finale = translation @ matrice_affine
                image1_transformee = cv2.warpAffine(image1, matrice_finale[:2], (x_max - x_min, y_max - y_min))
                scale_x = w / (x_max - x_min)
                scale_y = h / (y_max - y_min)
                image1_transformee = cv2.resize(image1_transformee, (w, h))
                ones = np.ones((points1.shape[0], 1), dtype=np.float32)
                points1_h = np.concatenate([points1, ones], axis=1)
                new_points1 = matrice_finale @ points1_h.T
                new_points1 = new_points1[:2, :].T
                new_points1[:, 0] *= scale_x
                new_points1[:, 1] *= scale_y

            else:
                print("Failed to calculate the affine transformation matrix.")
                return None, None
        else:
            print("Not enough key points for transformation.")
            return None, None

        return image1_transformee, new_points1

    def process_images_in_zones(self, image1, image2, list0, list0_2):
        h, w = image1.shape[:2]
        mid_x, mid_y = w // 2, h // 2
        list_image = []

        transformed_image = image1.copy()
        list_search = [(i + 1, tuple(point)) for i, point in enumerate(list0)]
        list_search_2 = [(i + 1, tuple(point)) for i, point in enumerate(list0_2)]
        lists_by_zone = {1: [], 2: [], 3: [], 4: []}

        for label, point in list_search:
            x, y = point
            if x < mid_x and y < mid_y:
                lists_by_zone[1].append(point)
            elif x >= mid_x and y < mid_y:
                lists_by_zone[2].append(point)
            elif x < mid_x and y >= mid_y:
                lists_by_zone[3].append(point)
            else:
                lists_by_zone[4].append(point)

        transformed_points = {}
        for i in range(1, 5):
            zone_points1 = lists_by_zone[i]

            if len(zone_points1) >= 3:
                zone_points2 = [list_search_2[label - 1][1] for label in range(1, len(list0) + 1) if tuple(list0[label - 1]) in zone_points1]

                if zone_points1:
                    points_zone1 = np.array(zone_points1, dtype=np.float32)
                    points_zone2 = np.array(zone_points2, dtype=np.float32)
                    transformed_zone, new_points_zone1 = self.affine_warping(transformed_image, points_zone1, points_zone2)
                    list_image.append(transformed_zone)
                    for point in zone_points1:
                        x, y = map(int, point)
                        transformed_image[y, x] = transformed_zone[y, x]
                    for j, (label, point) in enumerate(list_search):
                        if point in zone_points1:
                            transformed_points[label] = new_points_zone1[zone_points1.index(point)]
        final_list_search = [(label, transformed_points.get(label, point)) for label, point in list_search]
        return list_image,transformed_image, final_list_search

###########
new_curveconcatenationline= CurveConcatenationLine()
##########

In [ ]:
def undistorted(img):
    K = np.array([[1462.891043541062, 0.0, 1058.282670166303], [0.0, 1471.5340747924379, 590.6436713184269],[0.0, 0.0, 1.0]])
    D = np.array([[-0.005693882560794027],
                  [-0.27694517294013893],
                  [0.4672123487246388],
                  [-0.3332243527055097]])

    Knew = K.copy()
    Knew[(0, 1), (0, 1)] = 0.4 * Knew[(0, 1), (0, 1)]
    img_undistorted = cv2.fisheye.undistortImage(img, K, D=np.array([0., 0., 0., 0.]), Knew=K)
    return img_undistorted

def image_stitching(im1, im2):
  debut=time.time()
  im1_undistorted=im1
  im2_undistorted=im2
  height1,width1=im1_undistorted.shape[:2]
  print("Initiation of the appropriate zone selection...")
  mkpts_0, mkpts_1 = xfeat.match_xfeat(im1_undistorted, im2_undistorted, top_k = 4096, min_cossim=-1)
  new_curveconcatenationline.tracer_lignes_et_points_cv(im1_undistorted, mkpts_0)
  list_image,im1_undistorted,mkpts_0=new_curveconcatenationline.process_images_in_zones(im1_undistorted, im2_undistorted, mkpts_0, mkpts_1)
  mkpts_0 = np.array([item[1] for item in mkpts_0])
  category1, category2, category3, category4, category5, category6, category7,category8,category9,category10,category11,category12,category13,category14,category15,category16,category17,category18,category19,category20, summoy, nbre =0,0,0,0,0,0,0, 0, 0,0,0,0,0,0,0,0,0,0,0,0,0,0
  yaxis_class=[[0,100],[100,200],[200,300],[300,400],[400,500],[500,600],[600,700],[700,800],[800,900],[900,1000],[1000,1080]]
  xaxis_class=[[0,100],[100,200],[200,300],[300,400],[400,500],[500,600],[600,700],[700,800],[800,900],[900,950],[950,1000],[1000,1100],[1100,1200],[1200,1300],[1300,1400],[1400,1500],[1500,1600],[1600,1700],[1700,1800],[1800,1900],[1900,2000]]
  sum_result=None
  class_avgs=[]
  fixed_point=[]
  average_xaxis=[]
  average_yaxis=[]
  filtered_setleft,filtered_setright=[],[]
  k=0
  i,avg_abs,avg_ord1, avg_ord2,ydiff,bad_frame=0,0,0,0,0,0
  area,ydiff_left=[],[]
  concatpoint, mean_ratio=None,None
  set_points={}
  transformed=[]
  points_class,intensities_class=[],[]
  yleft_right,yright_left=0,0
  for y in range(len(mkpts_0)):
    if mkpts_0[y][0]>mkpts_1[y][0]:
      if mkpts_0[y][1]>mkpts_1[y][1]:
        filtered_setleft.append([mkpts_0[y],mkpts_1[y]])
        ydiff_left.append(mkpts_0[y][1]-mkpts_1[y][1])
        yleft_right+=1
      elif mkpts_0[y][1]<mkpts_1[y][1]:
        filtered_setright.append([mkpts_0[y],mkpts_1[y]])
        ydiff+=mkpts_1[y][1]-mkpts_0[y][1]
        yright_left+=1
  print("yleft_right:",yleft_right)
  print("yright_left:",yright_left)
  ydiff=ydiff/yright_left
  ydiff_left=statistics.mean(ydiff_left)

  filtered_setleft=[m for m in filtered_setleft if m[0][1]-m[1][1]<ydiff_left]
  filtered_setright=[m for m in filtered_setright if  m[1][1]-m[0][1]<ydiff]
  print("filtered_setleft:",filtered_setleft)

  if yleft_right>yright_left:
    liste1 = [ [pair[0][0], pair[1][0]] for pair in filtered_setleft ]
    liste2 = [ [pair[0][1], pair[1][1]] for pair in filtered_setleft ]
    for elt in xaxis_class:
      xdiff=[]
      class_points=[]
      summoy1,summoy,nbre=0,0,0
      for m,n in filtered_setleft:
        if m[0]>= elt[0] and m[0]<elt[1] :
          xdiff.append(m[0]-n[0])

      try:
        avgxdiff=statistics.mean(xdiff)
      except statistics.StatisticsError:
        avgxdiff=0
      average_xaxis.append([avgxdiff,len(xdiff)])

    data = [sublist[0] for sublist in average_xaxis if sublist[-1]>5]
    gen_avg=statistics.mean(data)

    diff_avg=[]
    for j in range(len(average_xaxis)-1):
      if average_xaxis[j][1]!=0 and average_xaxis[j+1][1]!=0:
        diff_avg.append([abs(average_xaxis[j][0] -average_xaxis[j+1][0]),j])

    n=len(data)
    var= sum((x - gen_avg) ** 2 for x in data) / n
    ecart=math.sqrt(var)
    small_diff,succ=[], ecart/2

    while len(small_diff)==0:
      small_diff=[num for num in diff_avg if num[0]<=succ and average_xaxis[num[1]][1]>5]
      succ+=5
    result = []
    current_group = []
    for element in small_diff:
      if not current_group or abs(element[1] - current_group[-1][1]) <= 1:
        current_group.append(element)
      else:
        result.append(current_group)
        current_group = [element]
    if current_group:
      result.append(current_group)
    diff_genavg=[]
    area= [[] for _ in range(len(result))]

    for e in range(len(result)):

      area[e].append(xaxis_class[result[e][0][1]][0])
      area[e].append(xaxis_class[result[e][-1][1]+1][1])

    diff_genavg=[]
    if len(area)>1:
      filtered_set=[[] for _ in range(len(result))]
      for i in range (len(area)):
        data=[m[0]-n[0] for m,n in filtered_setleft if area[i][0] <= m[0] <= area[i][1]]
        n = len(data)
        class_avg = statistics.mean(data)
        var= sum((x - class_avg) ** 2 for x in data) / n
        ecart=math.sqrt(var)
        print("area[i],class_avg,gen_avg,n,ecart", area[i],class_avg,gen_avg,n,ecart)
        diff_genavg.append([area[i],abs(class_avg-gen_avg)/n,[class_avg,ecart]])
        diff_genavg=sorted(diff_genavg, key=lambda x:x[1] )
      area=[m[0] for m in diff_genavg ]
      filtered_setcopy=[]
      for i in range (len(area)):
        error=diff_genavg[i][-1][1]/2
        class_avg=diff_genavg[i][-1][0]
        while len(filtered_set[i])==0 and error<diff_genavg[i][-1][1]/2+10:
          #if gen_avg-150 < class_avg < gen_avg+150:
          filtered_set[i]=[[m,n,m[0]-n[0]-class_avg] for m,n in filtered_setleft if area[i][0] <= m[0] <= area[i][1] and class_avg-error <m[0]-n[0]<class_avg+error]
          error+=5
        if len(filtered_set[i])>0:
          filtered_set[i]=sorted(filtered_set[i], key=lambda x:x[-1] )
          filtered_set[i]=[elt[:2] for elt in filtered_set[i]]
          filtered_setcopy.append(filtered_set[i])
          class_avgs.append(class_avg)
          set_points[str(area[i])]=filtered_set[i]
      try:
        mean_ratio=statistics.mean([class_avgs[0]-m for m in class_avgs[1:] if class_avgs[0]-m>0 and class_avgs[0]-m <=100])
      except statistics.StatisticsError:
        mean_ratio=None
      try:
        concatpoint=filtered_setcopy[0][int(len(filtered_setcopy)/2)][:2]
        filtered_setcopy_for_detection = [item for sublist in filtered_setcopy for item in sublist]
        index_greater_list =  new_curveconcatenationline.index_longest_list(filtered_setcopy)
        transformed = [[(point[0], point[1]) for point in sublist] for sublist in filtered_setcopy[index_greater_list]]
      except IndexError:
        print("No good keypoints found")

    else:
      filtered_set=[]
      data=[m[0]-n[0] for m,n in filtered_setleft if area[0][0] <= m[0] <= area[0][1]]
      n = len(data)
      class_avg = statistics.mean(data)
      var= sum((x - class_avg) ** 2 for x in data) / n
      ecart=math.sqrt(var)
      error=ecart/2
      while error<ecart/2+10:
        filtered_set=[[m,n,m[0]-n[0]-class_avg] for m,n in filtered_setleft if area[0][0] <= m[0] <= area[0][1] and class_avg-error <m[0]-n[0]<class_avg+error]
        error+=5

      class_avgs.append(class_avg)
      set_points[str(area[0])]=filtered_set

      try:
        filtered_set=sorted(filtered_set, key=lambda x:x[-1] )
        concatpoint=filtered_set[int(len(filtered_set)/2)][:2]
        transformed  = [[(float(point[0]), float(point[1])) for point in sublist if isinstance(point, np.ndarray) and point.shape == (2,)] for sublist in filtered_set]

      except IndexError:
        print("No good keypoints found")

  else:

    for elt in xaxis_class:
      xdiff=[]

      summoy1,summoy,nbre=0,0,0
      for m,n in filtered_setright:
        if elt[0]<= m[0]<elt[1] :

          xdiff.append(m[0]-n[0])
      try:
        avgxdiff=statistics.mean(xdiff)
        average_xaxis.append([avgxdiff,len(xdiff)])
      except statistics.StatisticsError:
        avgxdiff=0
        average_xaxis.append([avgxdiff,len(xdiff)])
    data = [sublist[0] for sublist in average_xaxis if sublist[-1]>5]
    gen_avg=statistics.mean(data)

    diff_avg=[]
    for j in range(len(average_xaxis)-1):
      if average_xaxis[j][1]!=0 and average_xaxis[j+1][1]!=0:
        diff_avg.append([abs(average_xaxis[j][0] -average_xaxis[j+1][0]),j])

    small_diff,succ=[], 10

    while len(small_diff)==0:
      small_diff=[num for num in diff_avg if num[0]<=succ and average_xaxis[num[1]][1]>5]
      succ+=5

    copysmall_diff=copy.deepcopy(small_diff)
    result = []
    current_group = []
    for element in small_diff:
      if not current_group or abs(element[1] - current_group[-1][1]) <= 1:
        current_group.append(element)
      else:
        result.append(current_group)
        current_group = [element]
    if current_group:
      result.append(current_group)

    area= [[] for _ in range(len(result))]
    diff_genavg=[]

    for e in range(len(result)):

      area[e].append(xaxis_class[result[e][0][1]][0])
      area[e].append(xaxis_class[result[e][-1][1]+1][1])

    diff_genavg=[]
    if len(area)>1:
      filtered_set=[[] for _ in range(len(result))]

      for i in range (len(area)):
        data=[m[0]-n[0] for m,n in filtered_setright if area[i][0] <= m[0] <= area[i][1]]
        n = len(data)
        class_avg = statistics.mean(data)
        var= sum((x - class_avg) ** 2 for x in data) / n
        ecart=math.sqrt(var)
        diff_genavg.append([area[i],abs(class_avg-gen_avg)/n,[class_avg,ecart]])
        diff_genavg=sorted(diff_genavg, key=lambda x:x[1] )
      area=[m[0] for m in diff_genavg ]
      class_avgs=[]
      filtered_setcopy=[]
      for i in range (len(area)):
        error=diff_genavg[i][-1][1]/2
        class_avg=diff_genavg[i][-1][0]

        while len(filtered_set[i])==0 and error<diff_genavg[i][-1][1]/2+10:

          filtered_set[i]=[[m,n,m[0]-n[0]-class_avg] for m,n in filtered_setright if area[i][0] <= m[0] <= area[i][1] and class_avg-error <m[0]-n[0]<class_avg+error]
          error+=5
        if len(filtered_set[i])>0:
          filtered_set[i]=sorted(filtered_set[i], key=lambda x:x[-1] )
          filtered_set[i]=[elt[:2] for elt in filtered_set[i]]
          filtered_setcopy.append(filtered_set[i])
          class_avgs.append(class_avg)
          set_points[str(area[i])]=filtered_set[i]
      try:
        mean_ratio=statistics.mean([class_avgs[0]-m for m in class_avgs[1:] if class_avgs[0]-m>0 and class_avgs[0]-m <=100])
      except statistics.StatisticsError:
        mean_ratio=None

      try:

        transformed = [[(point[0], point[1]) for point in sublist] for sublist in filtered_setcopy[0]]
      except IndexError:
        print("No good keypoints found")

    else:
      filtered_set=[]
      data=[m[0]-n[0] for m,n in filtered_setright if area[0][0] <= m[0] <= area[0][1]]
      n = len(data)
      class_avg = statistics.mean(data)
      var= sum((x - class_avg) ** 2 for x in data) / n
      ecart=math.sqrt(var)
      error=ecart/2
      while len(filtered_set)==0 and error<ecart/2+10:
        filtered_set=[[m,n,m[0]-n[0]-class_avg] for m,n in filtered_setright if area[0][0] <= m[0] <= area[0][1] and class_avg-error <m[0]-n[0]<class_avg+error]
        error+=5

      class_avgs.append(class_avg)
      set_points[str(area[0])]=filtered_set
      try:
        filtered_set=sorted(filtered_set, key=lambda x:x[-1] )
        concatpoint=filtered_set[int(len(filtered_set)/2)][:2]
        transformed  = [[(float(point[0]), float(point[1])) for point in sublist if isinstance(point, np.ndarray) and point.shape == (2,)] for sublist in filtered_set]

      except IndexError:
        print("No good keypoints found")

  filtered_list_transformed = []
  print("Initiation of selecting points to define the stitching line")
  current_list = transformed[0]
  filtered_list_transformed.append(current_list)

  for next_list in transformed[1:]:

      current_y = current_list[1][1]
      next_y = next_list[1][1]
      if next_y > current_y:
          filtered_list_transformed.append(next_list)
          current_list = next_list
  transformed=filtered_list_transformed

  matched_kp1 = [pair[0] for pair in transformed]
  matched_kp2 = [pair[1] for pair in transformed]
  intensities1,list_features_we_gets_intensity1,directions1,top_point1, bottom_point1=new_curveconcatenationline.get_scharr_intensities(im1,matched_kp1)
  intensities2,list_features_we_gets_intensity2,directions2,top_point2, bottom_point2=new_curveconcatenationline.get_scharr_intensities(im2,matched_kp2)
  differences = new_curveconcatenationline.compute_intensity_differences(intensities1, intensities2)
  matched_kp1, matched_kp2 = new_curveconcatenationline.filter_and_match(differences)
  latest_point, latest_point_corresponding,closest_left, corresponding_left, closest_top_left, corresponding_top_left, closest_right, corresponding_right,closest_bottom, corresponding_bottom,closest_bottom_right, corresponding_bottom_right =new_curveconcatenationline.find_closest_points(matched_kp1,matched_kp2)
  matched_kp1,matched_kp2=new_curveconcatenationline.remove_duplicates(matched_kp1,matched_kp2)
  matched_kp2, matched_kp1=new_curveconcatenationline.remove_duplicates(matched_kp2,matched_kp1)
  label_matched_kp1=new_curveconcatenationline.labelling_points(matched_kp1)
  label_matched_kp2=new_curveconcatenationline.labelling_points(matched_kp2)
  path_ids, path_points = new_curveconcatenationline.find_path(label_matched_kp1)
  points_img2_img1 = new_curveconcatenationline.getting_points_img2_corresponding(label_matched_kp2,path_ids)
  filtered_L2 = [points_img2_img1[0]]
  filtered_L1 = [path_points[0]]
  for i in range(1, len(points_img2_img1)):
      if filtered_L2[-1][1] < points_img2_img1[i][1]:
          filtered_L2.append(points_img2_img1[i])
          filtered_L1.append(path_points[i])
  points_img2_img1= filtered_L2
  path_points= filtered_L1
  triangles, connections =new_curveconcatenationline.generate_triangles(path_points)
  im3=im1_undistorted.copy()
  im4=im2_undistorted.copy()
  vertical_points1,img1, mask1,path_points1,check1 = new_curveconcatenationline.plot_lines_on_image(im1_undistorted,
                                                                                                    points=path_points,
                                                                                                    case = "im1")
  vertical_points2,img2, mask2,path_points2,check2 = new_curveconcatenationline.plot_lines_on_image(im2_undistorted,
                                                                                                    points=points_img2_img1,
                                                                                                    case = "im2")
  path_points1,path_points2=new_curveconcatenationline.filter_lists(path_points1,path_points2)
  segments1 = new_curveconcatenationline.horizontal_cut(im3, path_points1)
  for i, img in enumerate(segments1, start=1):
    filename = f"segments1-{i}.jpg"
    #cv2.imwrite(filename, img)
  segments2 = new_curveconcatenationline.horizontal_cut(im4, path_points2)
  for i, img in enumerate(segments2, start=1):
    filename = f"segments2-{i}.jpg"
    #cv2.imwrite(filename, img)
  list_dimensions1, list_points_adapt1 = new_curveconcatenationline.list_dimensions_list_points_adapt(segments1, path_points1)
  list_dimensions2, list_points_adapt2 = new_curveconcatenationline.list_dimensions_list_points_adapt(segments2, path_points2)
  result_horizontals,shift_first = new_curveconcatenationline.process_segments(segments1, segments2, list_points_adapt1, list_points_adapt2)
  adjusted_segments,result_vertical =new_curveconcatenationline.process_segmentsV(result_horizontals, matched_kp1)

  return result_vertical


In [ ]:
import os

def get_image_files(directory):
    return [os.path.join(directory, f) for f in sorted(os.listdir(directory)) if f.endswith(('.jpg', '.png'))]

dataset_dir = '/content/sample_data/datasets'
results_dir = '/content/sample_data/results'

os.makedirs(results_dir, exist_ok=True)

image_files = get_image_files(dataset_dir)

for i in range(0, len(image_files), 2):
    if i + 1 < len(image_files):
        im1 = cv2.imread(image_files[i])
        im2 = cv2.imread(image_files[i + 1])
        start_time = time.time()
        print(f"Image processing: {image_files[i]} and {image_files[i + 1]}")
        result_vertical = image_stitching(im1, im2)
        result_filename = f"stitched_{i//2}.jpg"
        cv2.imwrite(os.path.join(results_dir, result_filename), result_vertical)
        print(f"Saved image : {result_filename}")
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Running time for this pair: {execution_time:.4f} seconds")

Traitement des images : /content/sample_data/datasets/000336.jpg et /content/sample_data/datasets/000337.jpg
Initiation of the appropriate zone selection...


KeyboardInterrupt: 

In [ ]:
###################################

#####EVALUATION METRIC############
######################################
import threading
def evaluate_overlap_psnr_ssim(img1_warped, img2, mask1, mask2):

    h = min(img1_warped.shape[0], img2.shape[0], mask1.shape[0], mask2.shape[0])
    w = min(img1_warped.shape[1], img2.shape[1], mask1.shape[1], mask2.shape[1])

    img1_crop = img1_warped[:h, :w]
    img2_crop = img2[:h, :w]
    mask1_crop = mask1[:h, :w]
    mask2_crop = mask2[:h, :w]
    mask1_crop = mask1_crop[..., 0] if mask1_crop.ndim == 3 else mask1_crop
    mask2_crop = mask2_crop[..., 0] if mask2_crop.ndim == 3 else mask2_crop

    overlap_mask = (~mask1_crop) & (~mask2_crop)
    assert overlap_mask.ndim == 2, "overlap_mask must to be in 2D"

    overlap_mask_3d = np.repeat(overlap_mask[:, :, np.newaxis], 3, axis=2).astype(np.uint8)
    img1_overlap = (img1_crop * overlap_mask_3d).astype(np.uint8)
    img2_overlap = (img2_crop * overlap_mask_3d).astype(np.uint8)
    psnr = peak_signal_noise_ratio(img1_overlap, img2_overlap, data_range=255)
    ssim = structural_similarity(img1_overlap, img2_overlap, data_range=255, channel_axis=2)
    return psnr, ssim

def get_metrics_psnr_ssim(im1, im2):
  debut=time.time()
  im1_undistorted=im1
  im2_undistorted=im2
  height1,width1=im1_undistorted.shape[:2]

  mkpts_0, mkpts_1 = xfeat.match_xfeat(im1_undistorted, im2_undistorted, top_k = 4096, min_cossim=-1)
  new_curveconcatenationline.tracer_lignes_et_points_cv(im1_undistorted, mkpts_0)
  list_image,im1_undistorted,mkpts_0=new_curveconcatenationline.process_images_in_zones(im1_undistorted, im2_undistorted, mkpts_0, mkpts_1)
  mkpts_0 = np.array([item[1] for item in mkpts_0])
  category1, category2, category3, category4, category5, category6, category7,category8,category9,category10,category11,category12,category13,category14,category15,category16,category17,category18,category19,category20, summoy, nbre =0,0,0,0,0,0,0, 0, 0,0,0,0,0,0,0,0,0,0,0,0,0,0
  yaxis_class=[[0,100],[100,200],[200,300],[300,400],[400,500],[500,600],[600,700],[700,800],[800,900],[900,1000],[1000,1080]]
  xaxis_class=[[0,100],[100,200],[200,300],[300,400],[400,500],[500,600],[600,700],[700,800],[800,900],[900,950],[950,1000],[1000,1100],[1100,1200],[1200,1300],[1300,1400],[1400,1500],[1500,1600],[1600,1700],[1700,1800],[1800,1900],[1900,2000]]
  sum_result=None
  class_avgs=[]
  fixed_point=[]
  average_xaxis=[]
  average_yaxis=[]
  filtered_setleft,filtered_setright=[],[]
  k=0
  i,avg_abs,avg_ord1, avg_ord2,ydiff,bad_frame=0,0,0,0,0,0
  area,ydiff_left=[],[]
  concatpoint, mean_ratio=None,None
  set_points={}
  transformed=[]
  points_class,intensities_class=[],[]
  yleft_right,yright_left=0,0
  for y in range(len(mkpts_0)):
    if mkpts_0[y][0]>mkpts_1[y][0]:
      if mkpts_0[y][1]>mkpts_1[y][1]:
        filtered_setleft.append([mkpts_0[y],mkpts_1[y]])
        ydiff_left.append(mkpts_0[y][1]-mkpts_1[y][1])
        yleft_right+=1
      elif mkpts_0[y][1]<mkpts_1[y][1]:
        filtered_setright.append([mkpts_0[y],mkpts_1[y]])
        ydiff+=mkpts_1[y][1]-mkpts_0[y][1]
        yright_left+=1

  ydiff=ydiff/yright_left
  ydiff_left=statistics.mean(ydiff_left)

  filtered_setleft=[m for m in filtered_setleft if m[0][1]-m[1][1]<ydiff_left]
  filtered_setright=[m for m in filtered_setright if  m[1][1]-m[0][1]<ydiff]

  if yleft_right>yright_left:
    liste1 = [ [pair[0][0], pair[1][0]] for pair in filtered_setleft ]
    liste2 = [ [pair[0][1], pair[1][1]] for pair in filtered_setleft ]
    for elt in xaxis_class:
      xdiff=[]
      class_points=[]
      summoy1,summoy,nbre=0,0,0
      for m,n in filtered_setleft:
        if m[0]>= elt[0] and m[0]<elt[1] :
          xdiff.append(m[0]-n[0])

      try:
        avgxdiff=statistics.mean(xdiff)
      except statistics.StatisticsError:
        avgxdiff=0
      average_xaxis.append([avgxdiff,len(xdiff)])

    data = [sublist[0] for sublist in average_xaxis if sublist[-1]>5]
    gen_avg=statistics.mean(data)

    diff_avg=[]
    for j in range(len(average_xaxis)-1):
      if average_xaxis[j][1]!=0 and average_xaxis[j+1][1]!=0:
        diff_avg.append([abs(average_xaxis[j][0] -average_xaxis[j+1][0]),j])

    n=len(data)
    var= sum((x - gen_avg) ** 2 for x in data) / n
    ecart=math.sqrt(var)
    small_diff,succ=[], ecart/2

    while len(small_diff)==0:
      small_diff=[num for num in diff_avg if num[0]<=succ and average_xaxis[num[1]][1]>5]
      succ+=5
    result = []
    current_group = []
    for element in small_diff:
      if not current_group or abs(element[1] - current_group[-1][1]) <= 1:
        current_group.append(element)
      else:
        result.append(current_group)
        current_group = [element]
    if current_group:
      result.append(current_group)
    diff_genavg=[]
    area= [[] for _ in range(len(result))]

    for e in range(len(result)):

      area[e].append(xaxis_class[result[e][0][1]][0])
      area[e].append(xaxis_class[result[e][-1][1]+1][1])

    diff_genavg=[]
    if len(area)>1:
      filtered_set=[[] for _ in range(len(result))]
      for i in range (len(area)):
        data=[m[0]-n[0] for m,n in filtered_setleft if area[i][0] <= m[0] <= area[i][1]]
        n = len(data)
        class_avg = statistics.mean(data)
        var= sum((x - class_avg) ** 2 for x in data) / n
        ecart=math.sqrt(var)
        diff_genavg.append([area[i],abs(class_avg-gen_avg)/n,[class_avg,ecart]])
        diff_genavg=sorted(diff_genavg, key=lambda x:x[1] )
      area=[m[0] for m in diff_genavg ]
      filtered_setcopy=[]
      for i in range (len(area)):
        error=diff_genavg[i][-1][1]/2
        class_avg=diff_genavg[i][-1][0]
        while len(filtered_set[i])==0 and error<diff_genavg[i][-1][1]/2+10:
          #if gen_avg-150 < class_avg < gen_avg+150:
          filtered_set[i]=[[m,n,m[0]-n[0]-class_avg] for m,n in filtered_setleft if area[i][0] <= m[0] <= area[i][1] and class_avg-error <m[0]-n[0]<class_avg+error]
          error+=5
        if len(filtered_set[i])>0:
          filtered_set[i]=sorted(filtered_set[i], key=lambda x:x[-1] )
          filtered_set[i]=[elt[:2] for elt in filtered_set[i]]
          filtered_setcopy.append(filtered_set[i])
          class_avgs.append(class_avg)
          set_points[str(area[i])]=filtered_set[i]
      try:
        mean_ratio=statistics.mean([class_avgs[0]-m for m in class_avgs[1:] if class_avgs[0]-m>0 and class_avgs[0]-m <=100])
      except statistics.StatisticsError:
        mean_ratio=None
      try:
        concatpoint=filtered_setcopy[0][int(len(filtered_setcopy)/2)][:2]
        filtered_setcopy_for_detection = [item for sublist in filtered_setcopy for item in sublist]
        index_greater_list =  new_curveconcatenationline.index_longest_list(filtered_setcopy)
        transformed = [[(point[0], point[1]) for point in sublist] for sublist in filtered_setcopy[index_greater_list]]
      except IndexError:
        print("No good keypoints found")

    else:
      filtered_set=[]
      data=[m[0]-n[0] for m,n in filtered_setleft if area[0][0] <= m[0] <= area[0][1]]
      n = len(data)
      class_avg = statistics.mean(data)
      var= sum((x - class_avg) ** 2 for x in data) / n
      ecart=math.sqrt(var)
      error=ecart/2
      while error<ecart/2+10:
        filtered_set=[[m,n,m[0]-n[0]-class_avg] for m,n in filtered_setleft if area[0][0] <= m[0] <= area[0][1] and class_avg-error <m[0]-n[0]<class_avg+error]
        error+=5
      class_avgs.append(class_avg)
      set_points[str(area[0])]=filtered_set

      try:
        filtered_set=sorted(filtered_set, key=lambda x:x[-1] )
        concatpoint=filtered_set[int(len(filtered_set)/2)][:2]
        transformed  = [[(float(point[0]), float(point[1])) for point in sublist if isinstance(point, np.ndarray) and point.shape == (2,)] for sublist in filtered_set]

      except IndexError:
        print("No good keypoints found")

  else:

    for elt in xaxis_class:
      xdiff=[]

      summoy1,summoy,nbre=0,0,0
      for m,n in filtered_setright:
        if elt[0]<= m[0]<elt[1] :

          xdiff.append(m[0]-n[0])
      try:
        avgxdiff=statistics.mean(xdiff)
        average_xaxis.append([avgxdiff,len(xdiff)])
      except statistics.StatisticsError:
        avgxdiff=0
        average_xaxis.append([avgxdiff,len(xdiff)])
    data = [sublist[0] for sublist in average_xaxis if sublist[-1]>5]
    gen_avg=statistics.mean(data)

    diff_avg=[]
    for j in range(len(average_xaxis)-1):
      if average_xaxis[j][1]!=0 and average_xaxis[j+1][1]!=0:
        diff_avg.append([abs(average_xaxis[j][0] -average_xaxis[j+1][0]),j])

    small_diff,succ=[], 10

    while len(small_diff)==0:
      small_diff=[num for num in diff_avg if num[0]<=succ and average_xaxis[num[1]][1]>5]
      succ+=5

    copysmall_diff=copy.deepcopy(small_diff)
    result = []
    current_group = []
    for element in small_diff:
      if not current_group or abs(element[1] - current_group[-1][1]) <= 1:
        current_group.append(element)
      else:
        result.append(current_group)
        current_group = [element]
    if current_group:
      result.append(current_group)

    area= [[] for _ in range(len(result))]
    diff_genavg=[]

    for e in range(len(result)):

      area[e].append(xaxis_class[result[e][0][1]][0])
      area[e].append(xaxis_class[result[e][-1][1]+1][1])

    diff_genavg=[]
    if len(area)>1:
      filtered_set=[[] for _ in range(len(result))]

      for i in range (len(area)):
        data=[m[0]-n[0] for m,n in filtered_setright if area[i][0] <= m[0] <= area[i][1]]
        n = len(data)
        class_avg = statistics.mean(data)
        var= sum((x - class_avg) ** 2 for x in data) / n
        ecart=math.sqrt(var)
        diff_genavg.append([area[i],abs(class_avg-gen_avg)/n,[class_avg,ecart]])
        diff_genavg=sorted(diff_genavg, key=lambda x:x[1] )
      area=[m[0] for m in diff_genavg ]
      class_avgs=[]
      filtered_setcopy=[]
      for i in range (len(area)):
        error=diff_genavg[i][-1][1]/2
        class_avg=diff_genavg[i][-1][0]

        while len(filtered_set[i])==0 and error<diff_genavg[i][-1][1]/2+10:

          filtered_set[i]=[[m,n,m[0]-n[0]-class_avg] for m,n in filtered_setright if area[i][0] <= m[0] <= area[i][1] and class_avg-error <m[0]-n[0]<class_avg+error]
          error+=5
        if len(filtered_set[i])>0:
          filtered_set[i]=sorted(filtered_set[i], key=lambda x:x[-1] )
          filtered_set[i]=[elt[:2] for elt in filtered_set[i]]
          filtered_setcopy.append(filtered_set[i])
          class_avgs.append(class_avg)
          set_points[str(area[i])]=filtered_set[i]
      print("class_avgs",class_avgs)
      try:
        mean_ratio=statistics.mean([class_avgs[0]-m for m in class_avgs[1:] if class_avgs[0]-m>0 and class_avgs[0]-m <=100])
      except statistics.StatisticsError:
        mean_ratio=None

      try:

        transformed = [[(point[0], point[1]) for point in sublist] for sublist in filtered_setcopy[0]]
      except IndexError:
        print("No good keypoints found")

    else:
      filtered_set=[]
      data=[m[0]-n[0] for m,n in filtered_setright if area[0][0] <= m[0] <= area[0][1]]
      n = len(data)
      class_avg = statistics.mean(data)
      var= sum((x - class_avg) ** 2 for x in data) / n
      ecart=math.sqrt(var)
      error=ecart/2
      while len(filtered_set)==0 and error<ecart/2+10:
        filtered_set=[[m,n,m[0]-n[0]-class_avg] for m,n in filtered_setright if area[0][0] <= m[0] <= area[0][1] and class_avg-error <m[0]-n[0]<class_avg+error]
        error+=5

      class_avgs.append(class_avg)
      set_points[str(area[0])]=filtered_set
      try:
        filtered_set=sorted(filtered_set, key=lambda x:x[-1] )
        concatpoint=filtered_set[int(len(filtered_set)/2)][:2]
        transformed  = [[(float(point[0]), float(point[1])) for point in sublist if isinstance(point, np.ndarray) and point.shape == (2,)] for sublist in filtered_set]

      except IndexError:
        print("No good keypoints found")

  filtered_list_transformed = []
  current_list = transformed[0]
  filtered_list_transformed.append(current_list)

  for next_list in transformed[1:]:

      current_y = current_list[1][1]
      next_y = next_list[1][1]
      if next_y > current_y:
          filtered_list_transformed.append(next_list)
          current_list = next_list
  transformed=filtered_list_transformed

  matched_kp1 = [pair[0] for pair in transformed]
  matched_kp2 = [pair[1] for pair in transformed]
  intensities1,list_features_we_gets_intensity1,directions1,top_point1, bottom_point1=new_curveconcatenationline.get_scharr_intensities(im1,matched_kp1)
  intensities2,list_features_we_gets_intensity2,directions2,top_point2, bottom_point2=new_curveconcatenationline.get_scharr_intensities(im2,matched_kp2)
  differences = new_curveconcatenationline.compute_intensity_differences(intensities1, intensities2)
  matched_kp1, matched_kp2 = new_curveconcatenationline.filter_and_match(differences)
  latest_point, latest_point_corresponding,closest_left, corresponding_left, closest_top_left, corresponding_top_left, closest_right, corresponding_right,closest_bottom, corresponding_bottom,closest_bottom_right, corresponding_bottom_right =new_curveconcatenationline.find_closest_points(matched_kp1,matched_kp2)
  matched_kp1,matched_kp2=new_curveconcatenationline.remove_duplicates(matched_kp1,matched_kp2)
  matched_kp2, matched_kp1=new_curveconcatenationline.remove_duplicates(matched_kp2,matched_kp1)
  label_matched_kp1=new_curveconcatenationline.labelling_points(matched_kp1)
  label_matched_kp2=new_curveconcatenationline.labelling_points(matched_kp2)
  path_ids, path_points = new_curveconcatenationline.find_path(label_matched_kp1)
  points_img2_img1 = new_curveconcatenationline.getting_points_img2_corresponding(label_matched_kp2,path_ids)
  filtered_L2 = [points_img2_img1[0]]
  filtered_L1 = [path_points[0]]
  for i in range(1, len(points_img2_img1)):
      if filtered_L2[-1][1] < points_img2_img1[i][1]:
          filtered_L2.append(points_img2_img1[i])
          filtered_L1.append(path_points[i])
  points_img2_img1= filtered_L2
  path_points= filtered_L1
  triangles, connections =new_curveconcatenationline.generate_triangles(path_points)
  im3=im1_undistorted.copy()
  im4=im2_undistorted.copy()
  vertical_points1,img1, mask1,path_points1,check1 = new_curveconcatenationline.plot_lines_on_image(im1_undistorted,
                                                                                                    points=path_points,
                                                                                                    case = "im1")
  vertical_points2,img2, mask2,path_points2,check2 = new_curveconcatenationline.plot_lines_on_image(im2_undistorted,
                                                                                                    points=points_img2_img1,
                                                                                                    case = "im2")
  path_points1,path_points2=new_curveconcatenationline.filter_lists(path_points1,path_points2)

  segments1 = new_curveconcatenationline.horizontal_cut(im3, path_points1)
  segments2 = new_curveconcatenationline.horizontal_cut(im4, path_points2)
  list_dimensions1, list_points_adapt1 = new_curveconcatenationline.list_dimensions_list_points_adapt(segments1, path_points1)
  list_dimensions2, list_points_adapt2 = new_curveconcatenationline.list_dimensions_list_points_adapt(segments2, path_points2)
  result_horizontals,shift_first = new_curveconcatenationline.process_segments(segments1, segments2, list_points_adapt1, list_points_adapt2)
  bande = np.zeros((abs(shift_first), im1_undistorted.shape[1], 3), dtype=np.uint8)
  im3 = np.vstack((bande, im3)) if shift_first < 0 else np.vstack((im3, bande))
  mask1 = np.vstack((bande, mask1)) if shift_first < 0 else np.vstack((mask1, bande))
  im4 = np.vstack((bande, im4)) if shift_first > 0 else np.vstack((im4, bande))
  mask2 = np.vstack((bande, mask2)) if shift_first > 0 else np.vstack((mask2, bande))
  psnr, ssim =evaluate_overlap_psnr_ssim(im3, im4, mask1, mask2)
  return psnr, ssim


input1_dir = '/content/sample_data/testing/input1_sorted'
input2_dir = '/content/sample_data/testing/input2_sorted'


metrics_0_331 = []
metrics_332_663 = []
metrics_664_1106 = []

images1 = sorted(os.listdir(input1_dir), key=lambda x: int(os.path.splitext(x)[0]))
images2 = sorted(os.listdir(input2_dir), key=lambda x: int(os.path.splitext(x)[0]))
num_pairs = min(len(images1), len(images2))

start_time = time.time()
print("Start time:", start_time)


def run_with_timeout(func, timeout=180):
    result = {}
    def wrapper():
        try:
            result['value'] = func()
        except Exception as e:
            result['error'] = e

    thread = threading.Thread(target=wrapper)
    thread.start()
    thread.join(timeout)

    if thread.is_alive():
        return None, "Timeout"
    if 'error' in result:
        raise result['error']
    return result.get('value'), None

for i in range(num_pairs):
    filename = images1[i]
    base_index = int(os.path.splitext(filename)[0])

    if base_index > 1106:
        break

    im1_path = os.path.join(input1_dir, images1[i])
    im2_path = os.path.join(input2_dir, images2[i])

    im1 = cv2.imread(im1_path)
    im2 = cv2.imread(im2_path)

    if im1 is None or im2 is None:
        print(f"[SKIP] Image pair {images1[i]} or {images2[i]} not found or unreadable.")

        continue

    def stitch():
        return get_metrics_psnr_ssim(im1, im2)

    try:
        result, error = run_with_timeout(stitch, timeout=70)
        if error == "Timeout":

            continue

        psnr, ssim = result
        if psnr == float('inf'):

            continue
        if base_index <= 331:
            metrics_0_331.append((psnr, ssim))
        elif base_index <= 663:
            metrics_332_663.append((psnr, ssim))
        else:
            metrics_664_1106.append((psnr, ssim))

        print(f"[OK] Processed: {images1[i]} + {images2[i]} | PSNR: {psnr:.2f} | SSIM: {ssim:.4f}")

    except Exception as e:

        continue

end_time = time.time()
execution_time = end_time - start_time
print(f"\nTotal running time : {execution_time:.2f} seconds")

# Fonctions de moyenne avec sécurité
def safe_mean(metrics):
    if metrics:
        avg_psnr = np.mean([m[0] for m in metrics])
        avg_ssim = np.mean([m[1] for m in metrics])
        return avg_psnr, avg_ssim
    return None, None

# Affichage des résultats par plage
for name, metrics in [("0-331", metrics_0_331), ("332-663", metrics_332_663), ("664-1106", metrics_664_1106)]:
    avg_psnr, avg_ssim = safe_mean(metrics)
    if avg_psnr is not None:
        print(f"Mean PSNR for [{name}] : {avg_psnr:.2f}")
        print(f"Mean SSIM for [{name}] : {avg_ssim:.4f}")
    else:
        print(f"No metrics calculated for [{name}].")



Start time: 1747559583.1448185
Traitement des segments 0.................................................
segment1.shape: (508, 512, 3)
segment2.shape: (507, 512, 3)
Traitement des segments 1.................................................
segment1.shape: (4, 512, 3)
segment2.shape: (5, 512, 3)
[OK] Processed: 000001.jpg + 000001.jpg | PSNR: 30.15 | SSIM: 0.9981
Traitement des segments 0.................................................
segment1.shape: (327, 512, 3)
segment2.shape: (332, 512, 3)
Traitement des segments 1.................................................
segment1.shape: (185, 512, 3)
segment2.shape: (180, 512, 3)
[OK] Processed: 000002.jpg + 000002.jpg | PSNR: 25.06 | SSIM: 0.9927
Traitement des segments 0.................................................
segment1.shape: (268, 512, 3)
segment2.shape: (275, 512, 3)
Traitement des segments 1.................................................
segment1.shape: (244, 512, 3)
segment2.shape: (237, 512, 3)
[OK] Processed: 000003.jp

/usr/local/lib/python3.11/dist-packages/skimage/metrics/simple_metrics.py:168: RuntimeWarning: divide by zero encountered in scalar divide
  return 10 * np.log10((data_range**2) / err)


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
segment1.shape: (110, 512, 3)
segment2.shape: (110, 512, 3)
[SKIP] 000585.jpg + 000585.jpg: PSNR inf, skipping.
Traitement des segments 0.................................................
segment1.shape: (349, 512, 3)
segment2.shape: (349, 512, 3)
Traitement des segments 1.................................................
segment1.shape: (163, 512, 3)
segment2.shape: (163, 512, 3)
[SKIP] 000586.jpg + 000586.jpg: PSNR inf, skipping.
Traitement des segments 0.................................................
segment1.shape: (340, 512, 3)
segment2.shape: (355, 512, 3)
Traitement des segments 1.................................................
segment1.shape: (5, 512, 3)
segment2.shape: (7, 512, 3)
Traitement des segments 2.................................................
segment1.shape: (54, 512, 3)
segment2.shape: (54, 512, 3)
Traitement des segments 3.................................................
segment1.shape

# Nouvelle section